# Jonathan Halverson
# Thursday, May 18, 2017
# Pride and Prejudice

The text was taken from https://www.gutenberg.org/ebooks/1342

In [ ]:
with open('pride_and_prejudice_utf8_gutenberg.txt', 'r') as f:
     lines = f.readlines()
lines = lines[40:]

In [ ]:
lines = [line.decode('utf-8').replace('\r\n', '').encode('ascii', 'ignore') for line in lines]
lines

In [ ]:
lines = [line.replace('_', '') for line in lines if line]
lines

In [ ]:
lines = 
lines

Use a regular expression to remove numbers and punctuation:

In [ ]:
import re

lines = ' '.join(lines)
letters_only = re.sub('[^a-zA-Z]', ' ', lines)
words = letters_only.lower().split()

In [ ]:
len(words)

Need the NRC Word-Emotion Association Lexicon to continue following the ODSC post:
https://opendatascience.com/blog/you-must-allow-me-to-tell-you-how-ardently-i-admire-and-love-natural-language-processing/

The following link was used to obtain the lexicon:

https://github.com/aneesha/cbd/blob/master/cronscripts/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt

In [ ]:
from collections import OrderedDict, defaultdict, Counter
import csv

wordList = defaultdict(list)
emotionList = defaultdict(list)
with open('NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt', 'r') as f:
     reader = csv.reader(f, delimiter='\t')
     headerRows = [i for i in range(0, 46)]
     for row in headerRows:
          next(reader)
     for word, emotion, present in reader:
          if int(present) == 1:
               wordList[word].append(emotion)
               emotionList[emotion].append(word)

In [ ]:
wordList

In [ ]:
emotionList

In [ ]:
import pandas as pd

In [ ]:
iofile = 'NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt'
df = pd.read_csv(iofile, sep='\t', header=0, names=['word', 'emotion', 'value'])
df